## 문제5
공공데이터(data.go.kr)에서 <국민권익위원회 민원빅데이터 분석정보>

1. 시작일(2021.03.01.) ~ 종료일(2021.06.30.) 사이에 일반 민원 발생 건수가 많은 순서대로 정렬하여 json 형식으로 저장
2. 시작일(2021.01.01.) ~ 종료일(2021.12.31.) 사이에 월별 국가인권위원회의 일반민원, 고충민원을 csv 파일로 저장

In [1]:
from datetime import datetime, timedelta

def generate_date_period(first_date_str, last_date_str):
    # 날짜를 YYYYmmdd 형식으로 입력한다.
    first_date = datetime.strptime(first_date_str, '%Y%m%d')
    last_date = datetime.strptime(last_date_str, '%Y%m%d')
    date_str_generated = [datetime.strftime(first_date + timedelta(days=x), '%Y%m%d') for x in range(0, (last_date-first_date).days + 1)]
    return date_str_generated

#test
#generate_date_period('2021-03-01', '2021-06-30')

In [2]:
import requests
import json

api_key = 'U0%2Fs9G4kaV0wuxC5vNpgLfq%2Fm1S6NzGEs7U%2FJsTtSooG%2BfWfxHlxnMXnWRDE0Jo6mvmh62D%2FH3pqku9Dy7nWTQ%3D%3D'

# 교수님
#api_key = 'KoQvU3n5iWPhwcIINIlXksch6O5FI9%2Bpo7v5X4I2OTuEx5WmdZwPEdgH9zynJQMw0HH4NMoKC%2BcmrZv4z%2Br6QQ%3D%3D'

normal_minwon = 'pttn'
distress_minwon = 'dfpt'

human_right_committee_subcode = '1620000'

url_base = f'http://apis.data.go.kr/1140100/minAnalsInfoView3/minDFTopNKeyword3?serviceKey={api_key}&dataType=JSON'

def crawl_minwon(begin_date_str, month_count=1, minwon_type='pttn', subcode=None):
    url = f'{url_base}&target={minwon_type}&period=MONTHLY&analysisTime={begin_date_str}&rangeCount={month_count}&maxResult=10000'
    if subcode:
        url += f'&mainSubCode={subcode}'
    
    res = requests.get(url)
    if res.status_code != 200:
        print(f'error {res.status_code}')
        return None

    content = res.content.decode('utf-8').strip()
    if not content or 'ERROR' in content:
        print(f'ERROR: {begin_date_str}')
        return None
    
    return json.loads(content)

# test
#print(json.dumps(crawl_minwon('20210301'), ensure_ascii=False, indent=2))


In [3]:
# (1) 시작일(20210301) ~ 종료일(20210630) 사이에 일반민원 발생 건수 많은 순서대로 정렬 --> json
import json

begin_date = '20210301'
minwon_j = crawl_minwon(begin_date, month_count=4)

with open('answer5-1.json', 'w', encoding='utf-8') as f:
    json.dump(minwon_j, f, ensure_ascii=False, indent=2)


In [4]:
# (2) 시작일(20200101) ~ 종료일(20211231) 사이에 월별 일반민원, 고충민원 --> csv
import csv
from tqdm.notebook import tqdm
import json

rows = []
for y in range(2):
    for m in range(1, 12+1):
        date_str = f'202{y}{m:02d}01'
        for k, v in {'일반민원': normal_minwon, '고충민원': distress_minwon}.items():
            j = crawl_minwon(date_str, month_count=1, minwon_type=v, subcode=human_right_committee_subcode)
            if not j:
                print(f'{k} {date_str} 없음')
                continue

            for item in j:
                rows.append([f'202{y}-{m:02d}', k, item['term'], item['df']])

with open(f'answer5-2.csv', 'w', encoding='utf-8', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(rows)


고충민원 20200101 없음
고충민원 20200201 없음
고충민원 20200301 없음
고충민원 20200401 없음
고충민원 20200501 없음
고충민원 20200601 없음
고충민원 20200701 없음
고충민원 20200801 없음
고충민원 20200901 없음
고충민원 20201001 없음
고충민원 20201101 없음
고충민원 20201201 없음
고충민원 20210101 없음
고충민원 20210201 없음
고충민원 20210301 없음
고충민원 20210401 없음
고충민원 20210501 없음
고충민원 20210601 없음
고충민원 20210701 없음
고충민원 20210801 없음
고충민원 20210901 없음
고충민원 20211001 없음
고충민원 20211101 없음
일반민원 20211201 없음
고충민원 20211201 없음
